In [52]:
import pandas as pd

In [105]:
mvp_voting_df : pd.DataFrame= pd.read_csv('../data/mvp_voting_1991-2022.csv')
# Remove player stat data from the MVP voting data 
# since we already have this information in the player stats csv
voting_data_cols = ['Player', 'Year', 'Pts Won', 'Pts Max', 'Share']
mvp_voting_df = mvp_voting_df[voting_data_cols]

In [148]:
player_stats_df : pd.DataFrame = pd.read_csv('../data/player_stats_1991-2022.csv')
# Delete extraneous columns from the player data
del player_stats_df['Rk']
del player_stats_df['Unnamed: 0']
# Delete any asterisks on player names for successful combination
player_stats_df['Player'] = player_stats_df['Player'].str.replace('*','',regex=False)

# There are multiple entries for players that play for multiple teams into a single entry
player_stats_df.groupby(['Player','Year']).get_group(('Greg Anderson',1991)) 

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
8,Greg Anderson,PF,26,TOT,68,2,13.6,1.7,4.0,0.43,...,1.4,3.3,4.7,0.2,0.5,0.7,1.2,2.1,4.3,1991
9,Greg Anderson,PF,26,MIL,26,0,9.5,1.0,2.8,0.37,...,1.0,1.9,2.9,0.1,0.3,0.3,0.8,1.1,2.7,1991
10,Greg Anderson,PF,26,NJN,1,0,18.0,4.0,4.0,1.00,...,4.0,2.0,6.0,1.0,2.0,0.0,1.0,4.0,8.0,1991
11,Greg Anderson,PF,26,DEN,41,2,16.1,2.1,4.7,0.44,...,1.6,4.1,5.8,0.3,0.6,0.9,1.5,2.6,5.2,1991


In [139]:
# Combine the grouped data so that each player only has one entry per season 
def single_row(df : pd.DataFrame):
    if df.shape[0] == 1: # If there is only one entry, we are all set
        return df
    else: # If there are multiple entries, the entry for the team 'TOT' has the player's total stats
        row = df[df['Tm'] == 'TOT'] # Use the TOT entry for the total stats
        # Replace the team TOT with the player's most recent team 
        row['Tm'] = df.iloc[-1, :]['Tm'] 
        return row

player_stats_df = player_stats_df.groupby(['Player', 'Year']).apply(single_row)

In [140]:
# Drop the multi-indexes that were created with the groupby() function
player_stats_df.index = player_stats_df.index.droplevel()
player_stats_df.index = player_stats_df.index.droplevel()



In [141]:
player_stats_df[player_stats_df['Player'] == 'Greg Anderson']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
8,Greg Anderson,PF,26,DEN,68,2,13.6,1.7,4.0,0.430,...,1.4,3.3,4.7,0.2,0.5,0.7,1.2,2.1,4.3,1991
449,Greg Anderson,PF,27,DEN,82,82,34.1,4.7,10.4,0.456,...,4.1,7.4,11.5,1.0,1.1,0.8,2.5,3.2,11.5,1992
1358,Greg Anderson,PF,29,DET,77,47,21.1,2.6,4.8,0.543,...,2.4,5.0,7.4,0.7,0.7,0.9,1.2,3.0,6.4,1994
1838,Greg Anderson,PF,30,ATL,51,0,12.2,1.1,2.0,0.548,...,1.2,2.5,3.7,0.3,0.5,0.6,0.6,2.0,2.9,1995
2289,Greg Anderson,PF,31,SAS,46,7,7.5,0.5,1.0,0.511,...,0.6,1.5,2.2,0.2,0.2,0.5,0.5,1.4,1.2,1996
2836,Greg Anderson,C,32,SAS,82,48,20.2,1.6,3.2,0.496,...,1.9,3.5,5.5,0.4,0.8,0.8,0.9,2.7,3.9,1997
3408,Greg Anderson,C,33,ATL,50,0,8.0,0.7,1.6,0.444,...,0.8,1.6,2.4,0.3,0.4,0.2,0.3,1.7,1.8,1998


In [142]:
# Merge the player stat and MVP voting data into a single dataframe
# Do an outer merge, since the player data has all player data (even those that got 0 mvp votes)
full_player_df = player_stats_df.merge(mvp_voting_df, how='outer',on=['Player','Year'])
# Check to see if the merge properly carried over players who received votes
full_player_df.head(20)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,0.9,0.7,0.3,1.2,1.4,9.1,1991,NaN,NaN,NaN
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,0.476,...,1.4,1.1,0.4,1.4,1.7,13.6,1992,NaN,NaN,NaN
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,0.537,...,1.4,1.1,0.5,1.4,1.8,12.8,1993,NaN,NaN,NaN
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,0.502,...,1.7,0.9,0.5,1.2,1.7,14.7,1994,NaN,NaN,NaN
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,1.5,0.7,0.4,1.4,1.8,11.2,1995,NaN,NaN,NaN
5,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,0.484,...,0.9,0.5,0.3,1.0,1.7,7.5,1996,NaN,NaN,NaN
6,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,0.483,...,0.8,0.8,0.2,0.9,1.7,7.2,1997,NaN,NaN,NaN
7,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,0.453,...,1.5,1.0,0.3,0.8,1.9,7.3,1998,NaN,NaN,NaN
8,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,0.422,...,0.5,0.6,0.2,0.4,1.4,4.9,1999,NaN,NaN,NaN
9,A.C. Green,PF,36,LAL,82,82,23.5,2.1,4.7,0.447,...,1.0,0.6,0.2,0.6,1.5,5.0,2000,NaN,NaN,NaN


In [143]:
# Fill NaN values denoting 0 MVP votes won with 0s
full_player_df[['Pts Won','Pts Max','Share']] = full_player_df[['Pts Won','Pts Max','Share']].fillna(0)

In [127]:
# Read in the team stats DF for cleaning
team_data_df : pd.DataFrame = pd.read_csv('../data/team_data_1991-2022.csv')
# Delete any asterisks on player names for successful combination
team_data_df['Team'] = team_data_df['Team'].str.replace('*','',regex=False)
team_data_df['Team'].unique()

array(['Boston Celtics', 'Philadelphia 76ers', 'New York Knicks',
       'Washington Bullets', 'New Jersey Nets', 'Miami Heat',
       'Chicago Bulls', 'Detroit Pistons', 'Milwaukee Bucks',
       'Atlanta Hawks', 'Indiana Pacers', 'Cleveland Cavaliers',
       'Charlotte Hornets', 'San Antonio Spurs', 'Utah Jazz',
       'Houston Rockets', 'Orlando Magic', 'Minnesota Timberwolves',
       'Dallas Mavericks', 'Denver Nuggets', 'Portland Trail Blazers',
       'Los Angeles Lakers', 'Phoenix Suns', 'Golden State Warriors',
       'Seattle SuperSonics', 'Los Angeles Clippers', 'Sacramento Kings',
       'Toronto Raptors', 'Vancouver Grizzlies', 'Washington Wizards',
       'Memphis Grizzlies', 'New Orleans Hornets', 'Charlotte Bobcats',
       'New Orleans/Oklahoma City Hornets', 'Oklahoma City Thunder',
       'Brooklyn Nets', 'New Orleans Pelicans'], dtype=object)

In [144]:
full_player_df['Tm'].unique()

array(['LAL', 'PHO', 'DAL', 'MIA', 'CLE', 'WSB', 'CHI', 'GSW', 'IND',
       'WAS', 'MIN', 'BOS', 'HOU', 'DEN', 'ORL', 'NOH', 'TOR', 'SAC',
       'CHO', 'PHI', 'POR', 'DET', 'OKC', 'UTA', 'MIL', 'VAN', 'SEA',
       'NJN', 'NOK', 'LAC', 'ATL', 'CHA', 'MEM', 'NYK', 'NOP', 'BRK',
       'SAS', 'CHH'], dtype=object)

In [145]:
nicknames = {}

with open('nicknames.csv') as f: 
    lines = f.readlines()
    for line in lines[1:]: # Skip the first line that describes the CSV
         # Remove the new line from the CSV
        # Split the line into the nickname : name mapping ex. ATL : Atlanta Hawks
        abbrev, name = line.replace('\n','').split(',')
        nicknames[abbrev] = name
nicknames

{'ATL': 'Atlanta Hawks',
 'BRK': 'Brooklyn Nets',
 'BKN': 'Brooklyn Nets',
 'BOS': 'Boston Celtics',
 'CHA': 'Charlotte Bobcats',
 'CHH': 'Charlotte Hornets',
 'CHO': 'Charlotte Hornets',
 'CHI': 'Chicago Bulls',
 'CLE': 'Cleveland Cavaliers',
 'DAL': 'Dallas Mavericks',
 'DEN': 'Denver Nuggets',
 'DET': 'Detroit Pistons',
 'GSW': 'Golden State Warriors',
 'HOU': 'Houston Rockets',
 'IND': 'Indiana Pacers',
 'LAC': 'Los Angeles Clippers',
 'LAL': 'Los Angeles Lakers',
 'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',
 'MIL': 'Milwaukee Bucks',
 'MIN': 'Minnesota Timberwolves',
 'NJN': 'New Jersey Nets',
 'NOH': 'New Orleans Hornets',
 'NOP': 'New Orleans Pelicans',
 'NOK': 'New Orleans/Oklahoma City Hornets',
 'NYK': 'New York Knicks',
 'OKC': 'Oklahoma City Thunder',
 'ORL': 'Orlando Magic',
 'PHI': 'Philadelphia 76ers',
 'PHX': 'Phoenix Suns',
 'PHO': 'Phoenix Suns',
 'POR': 'Portland Trail Blazers',
 'SEA': 'Seattle SuperSonics',
 'SAC': 'Sacramento Kings',
 'SAS': 'San Antonio Spu

In [146]:
full_player_df['Team'] = full_player_df['Tm'].map(nicknames)
full_player_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share,Team
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,0.7,0.3,1.2,1.4,9.1,1991,0.0,0.0,0.0,Los Angeles Lakers
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,0.476,...,1.1,0.4,1.4,1.7,13.6,1992,0.0,0.0,0.0,Los Angeles Lakers
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,0.537,...,1.1,0.5,1.4,1.8,12.8,1993,0.0,0.0,0.0,Los Angeles Lakers
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,0.502,...,0.9,0.5,1.2,1.7,14.7,1994,0.0,0.0,0.0,Phoenix Suns
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0,Phoenix Suns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,0.505,...,0.4,1.0,1.1,2.6,6.9,2002,0.0,0.0,0.0,Detroit Pistons
14693,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,0.552,...,0.2,0.6,1.0,2.6,6.6,2003,0.0,0.0,0.0,Detroit Pistons
14694,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,0.442,...,0.2,0.5,0.7,2.2,3.8,2004,0.0,0.0,0.0,Atlanta Hawks
14695,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,0.568,...,0.2,0.7,0.8,2.2,5.8,2005,0.0,0.0,0.0,Los Angeles Clippers


In [151]:
# Merge our player stats DF with the team data frame
team_and_stats_df = full_player_df.merge(team_data_df, how = 'outer', on=['Team', 'Year'])
del team_and_stats_df['Unnamed: 0']
team_and_stats_df[team_and_stats_df['Pts Won'] > 0]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
6,Magic Johnson,PG,31,LAL,79,79,37.1,5.9,12.4,0.477,...,960.0,0.518,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
42,Charles Barkley,PF,30,PHO,65,65,35.4,8.0,16.1,0.495,...,1010.0,0.005,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
50,Kevin Johnson,PG,27,PHO,67,67,36.6,7.1,14.6,0.487,...,1010.0,0.001,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
58,Charles Barkley,PF,31,PHO,68,66,35.0,8.1,16.8,0.486,...,1050.0,0.091,Phoenix Suns,59,23,0.720,—,110.6,106.8,3.86
71,Charles Barkley,PF,32,PHO,71,71,37.1,8.2,16.3,0.500,...,1130.0,0.007,Phoenix Suns,41,41,0.500,23.0,104.3,104.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14514,Larry Bird,SF,35,BOS,45,45,36.9,7.8,16.8,0.466,...,960.0,0.003,Boston Celtics,51,31,0.622,—,106.6,103.0,3.41
14548,Shaquille O'Neal,C,26,LAL,49,49,34.8,10.4,18.1,0.576,...,1180.0,0.075,Los Angeles Lakers,31,19,0.620,4.0,99.0,96.0,2.68
14607,Dominique Wilkins,SF,31,ATL,81,81,38.0,9.5,20.2,0.470,...,960.0,0.030,Atlanta Hawks,43,39,0.524,18.0,109.8,109.0,0.72
14623,Jason Kidd,PG,21,DAL,79,79,33.8,4.2,10.8,0.385,...,1050.0,0.007,Dallas Mavericks,36,46,0.439,26.0,103.2,106.1,-2.39
